# Notebook for Creating and Training DL Model for PV Power Forecasting

### Contents:
- Imports and Definitions
- Hyperparameter Definitions
- Data Loader
- Choose Model Name
- Choose Model Architecture
- Train and Evaluate the Model
- Test Model against Benchmark Model


In [1]:
# Importing libraries
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import kurtosis, skew
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense, Concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16

from helper_functions_model import *



from google.colab import drive
drive.mount('/content/drive')

# Define Paths
BASE_FILE_PATH = "..."
SAVE_MODEL_PATH = "..."

print("Setup complete!")


Mounted at /content/drive
Setup complete!


# Load preprozessed train/Test Dataset  

In [ ]:
# Load preprozessed Train-/ Testset

folder= os.path.join(BASE_FILE_PATH,"...")
data_set = load_preprocessed_data(folder, small_dataset=False)

folder= os.path.join(BASE_FILE_PATH,"...")
aux_data = load_preprocessed_data(folder, small_dataset=False)

print_shapes(data_set = data_set)
print_shapes(aux_data = aux_data)


# Choose a Model and Name

In [4]:
# CHANGE MODEL NAME!
model_name = "Temp_V3.2"

# 1. Image Only


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU
from tensorflow.keras.models import Model

# Define input layers
x_in = Input(shape=(64, 64, 3))
x2_in = Input(shape=(16,))
x3_in = Input(shape=(1,))

# CNN image processing
x = Conv2D(32, (3, 3), padding='same')(x_in)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

x = Dense(1024)(x)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.35)(x)
x = Dense(512)(x)
x = LeakyReLU(alpha=0.01)(x)

x = Dense(1, activation='linear')(x)

# Create the model
model = Model(inputs=[x_in, x2_in, x3_in], outputs=x)

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 64, 64, 32)           896       ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)           0         ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)           0         ['leaky_re_lu[0][0]']         
 D)                                                                                           

# 2. Only Numarical PV-log Data


In [ ]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout
from tensorflow.keras.models import Model

# Define input layers
x_in = Input(shape=(64, 64, 3))
x2_in = Input(shape=(16,))
x3_in = Input(shape=(1,))

# Fully Connected Layer for the second input data
x2 = Dense(48)(x2_in)
x2 = LeakyReLU(alpha=0.01)(x2)
x2 = Dense(48)(x2)
x2 = LeakyReLU(alpha=0.01)(x2)

x = Dense(1024)(x2)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.35)(x)
x = Dense(512)(x)
x = LeakyReLU(alpha=0.01)(x)

x = Dense(1, activation='linear')(x)

# Create the model
model = Model(inputs=[x_in, x2_in, x3_in], outputs=x)

# Print model summary
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 dense_8 (Dense)             (None, 48)                   816       ['input_8[0][0]']             
                                                                                                  
 leaky_re_lu_9 (LeakyReLU)   (None, 48)                   0         ['dense_8[0][0]']             
                                                                                                  
 dense_9 (Dense)             (None, 48)                   2352      ['leaky_re_lu_9[0][0]']       
                                                                                            

# 3. Numarical PV-log Data AND SPM Prediction


In [ ]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, Add
from tensorflow.keras.models import Model

# Define input layers
x_in = Input(shape=(64, 64, 3))
x2_in = Input(shape=(16,))
x3_in = Input(shape=(1,))

# Fully Connected Layer for the second input data
x2 = Dense(48)(x2_in)
x2 = LeakyReLU(alpha=0.01)(x2)
x2 = Dense(48)(x2)
x2 = LeakyReLU(alpha=0.01)(x2)

x = Dense(1024)(x2)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.35)(x)
x = Dense(512)(x)
x = LeakyReLU(alpha=0.01)(x)

x = Dense(1, activation='linear')(x)

x = Add()([x, x3_in])

x = Dense(1, activation='linear')(x)

# Create the model
model = Model(inputs=[x_in, x2_in, x3_in], outputs=x)

# Print model summary
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 dense_8 (Dense)             (None, 48)                   816       ['input_8[0][0]']             
                                                                                                  
 leaky_re_lu_9 (LeakyReLU)   (None, 48)                   0         ['dense_8[0][0]']             
                                                                                                  
 dense_9 (Dense)             (None, 48)                   2352      ['leaky_re_lu_9[0][0]']       
                                                                                            

# 4. Image AND SPM Predictions Only


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Add
from tensorflow.keras.models import Model

# Define input layers
x_in = Input(shape=(64, 64, 3))
x2_in = Input(shape=(16,))
x3_in = Input(shape=(1,))

# CNN image processing
x = Conv2D(32, (3, 3), padding='same')(x_in)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

x = Dense(1024)(x)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.35)(x)
x = Dense(512)(x)
x = LeakyReLU(alpha=0.01)(x)

x = Dense(1, activation='linear')(x)

x = Add()([x, x3_in])

x = Dense(1, activation='linear')(x)

# Create the model
model = Model(inputs=[x_in, x2_in, x3_in], outputs=x)

# Print model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_3 (Conv2D)           (None, 64, 64, 32)           896       ['input_4[0][0]']             
                                                                                                  
 leaky_re_lu_7 (LeakyReLU)   (None, 64, 64, 32)           0         ['conv2d_3[0][0]']            
                                                                                                  
 max_pooling2d_3 (MaxPoolin  (None, 32, 32, 32)           0         ['leaky_re_lu_7[0][0]']       
 g2D)                                                                                       

# 5. Imag and PV log


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Concatenate
from tensorflow.keras.models import Model

# Define input layers
x_in = Input(shape=(64, 64, 3))
x2_in = Input(shape=(16,))
x3_in = Input(shape=(1,))

# CNN image processing
x = Conv2D(32, (3, 3), padding='same')(x_in)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

# Fully Connected Layer for the second input data
x2 = Dense(48)(x2_in)
x2 = LeakyReLU(alpha=0.01)(x2)
x2 = Dense(48)(x2)
x2 = LeakyReLU(alpha=0.01)(x2)

# Concatenate the RNN output with the previous model
concatenated = Concatenate()([x, x2])

x = Dense(1024)(concatenated)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.35)(x)
x = Dense(512)(x)
x = LeakyReLU(alpha=0.01)(x)

x = Dense(1, activation='linear')(x)

# Create the model
model = Model(inputs=[x_in, x2_in, x3_in], outputs=x)

# Print model summary
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 64, 64, 32)           896       ['input_10[0][0]']            
                                                                                                  
 leaky_re_lu_12 (LeakyReLU)  (None, 64, 64, 32)           0         ['conv2d_6[0][0]']            
                                                                                                  
 max_pooling2d_6 (MaxPoolin  (None, 32, 32, 32)           0         ['leaky_re_lu_12[0][0]']      
 g2D)                                                                                       

# 6. Full Proposed DL Prediction Modell


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Concatenate, Add
from tensorflow.keras.models import Model

# Define input layers
x_in = Input(shape=(64, 64, 3))
x2_in = Input(shape=(16,))
x3_in = Input(shape=(1,))

# CNN image processing
x = Conv2D(32, (3, 3), padding='same')(x_in)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), padding='same')(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

# Fully Connected Layer for the second input data
x2 = Dense(48)(x2_in)
x2 = LeakyReLU(alpha=0.01)(x2)
x2 = Dense(48)(x2)
x2 = LeakyReLU(alpha=0.01)(x2)

# Concatenate the RNN output with the previous model
concatenated = Concatenate()([x, x2])

x = Dense(1024)(concatenated)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.35)(x)
x = Dense(512)(x)
x = LeakyReLU(alpha=0.01)(x)

x = Dense(1, activation='linear')(x)

x = Add()([x, x3_in])

x = Dense(1, activation='linear')(x)

# Create the model
model = Model(inputs=[x_in, x2_in, x3_in], outputs=x)

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 64, 64, 32)           896       ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)           0         ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)           0         ['leaky_re_lu[0][0]']         
 D)                                                                                           

# Train the Model

In [ ]:
# Define Hyperparameters:

hyper_params = {
    "batch_size": 96,
    "num_epochs": 40,
    "learning_rate": 0.0001
}


In [ ]:
# Train and save the model

model, history = train_my_model(model=model,
                                model_name=model_name,
                                train_input=[data_set["images_train"] ,aux_data["sequence_labels_train"][:,:-1], aux_data["sequence_labels_train"][:,-1:]], # needs to be changed, deppending on model Architekture
                                train_labels=data_set["labels_train"],
                                val_input=[data_set["images_validation"], aux_data["sequence_labels_validation"][:,:-1] ,aux_data["sequence_labels_validation"][:,-1:]],     # needs to be changed, deppending on model Architekture
                                val_labels=data_set["labels_validation"],
                                early_stopping_patience=2,
                                opti="Adam",  # Options: "Adam" or "SGD"
                                loss_function='mean_squared_error', # Options: 'mean_squared_error', 'mean_absolute_error'
                                )


# Evaluate the Model

In [5]:
# load a alredy traind model with wights:
model = load_my_model(model_name="...")


In [ ]:
model.summary()

In [ ]:
# choose a timeframe for a zoomed plot
start_timestamp = "2019-05-27 14:00:00"
end_timestamp = "2019-05-27 14:30:00"



evaluate_model_performance(model=model,
                           model_name=model_name,
                           test_input=[data_set["images_test"], aux_data["sequence_labels_test"][:,:-1] , aux_data["sequence_labels_test"][:,-1:]], # needs to be changed, deppending on model Architekture
                           test_labels=data_set["labels_test"],
                           test_timestamps=data_set["datetime_test"],
                           small_dataset=False,
                           spm_path="...",
                           start_timestamp=start_timestamp,
                           end_timestamp=end_timestamp)

